# Link to Video


# Introduction
This project is based on "Back to the Basics with Inclusion of Clinical Domain Knowledge - A Simple, Scalable and Effective Model of Alzheimer’s Disease Classification" (Sarah C. Brüningk, Felix Hensel, Louis P. Lukas, Merel Kuijs, Catherine R. Jutzeler, Bastian Rieck Proceedings of the 6th Machine Learning for Healthcare Conference, PMLR 149:730-754, 2021.)

The authors of the original paper analyze MRIs of patients with Alzheimer's disease to detect the onset of the disease early, predict disease progression, and optimize treatment plans for patients.

Currently, there is a push for more complex machine learning models, like deep CNNs with unsupervised learning and ensemble models that combine different architectures. Complex models may perform well, but they are not necessarily applicable in clinical settings because high-resolution MRIs lead to memory and hardware issues. One method used to achieve high performance is down-sampling data, but this means that high-resolution data that could be useful is scrapped.

The authors hypothesize that using simple models that include clinical domain knowledge of Alzheimer's disease and its topological features will result in a classification performance similar to that of complex models while also being faster, more scalable, and more interpretable. They evaluated the performance of CNNs on subsets of full-resolution three-dimensional MRIs. Then, they performed an ablation study to compare different levels of biological scale and anatomical brain areas. Finally, they used topological data analysis (TDA) to analyze changes in whole brain tissue connectivity.

They found that choosing a meaningful data representation consisting of the left hippocampus allowed them to achieve better performance compared to more complex architectures with many parameters. In addition, they were able to save on computational cost and hardware requirements. This means that clinical domain knowledge may be more important than architecture design and complexity, specifically for Alzheimer's Disease classification.

# Scope of Reproducibility

The authors' hypotheses were
1. "...using simple model architectures that include prior domain knowledge of morphological hallmarks of [Alzheimer's Disease] and topological feature thereof, will result in classification performance comparable to more complex model[s]" (3), and
2. "...[their] models will facilitate fast, scalable, and interpretable solutions through meaningful representations" (3).

In other words, they sought to determine if simple models are faster, more usable, and more understandable than highly complex models with too many hyperparameters.

In this project, I use a simple CNN model, using HW 3: CNNs as a guide, to test if a basic CNN will have a high accuracy (>= 70%) and perform fast.


# Methodology

## Environment

Python version: 3.10

## Package Imports

In [ ]:
import numpy as np
from numpy import mean
import os
import random
import pickle
import pandas as pd

import matplotlib.pyplot as plt

import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import tensorflow as tf

from tqdm import tqdm

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import KFold

import gdown

##  Data
The data for this project was obtained from the Alzheimer's Disease Neuroimaging Initiative (ADNI) database. The original experiment used a sample of 358 subjects, which included both control and Alzheimer's Disease patients. The mean age was 77 +/- 7 years. Scans were acquired each year, up to 10 years, for each patient.

The GitHub repository contains a folder with partitions used for five fold cross validation with training, validation, and test sets. For training, all longitudinal scans of a particular patient were used. For validation and testing, only a single randomly selected scan was used.

I only used a subset of the data since each patient had multiple photos, which would make the total number of photos too high for this project.

## Download instructions
 1. Request access to the ADNI dataset (https://ida.loni.usc.edu/collaboration/access/appLicense.jsp)
 2. Once you receive an email confirming your access, login with your credentials here: https://ida.loni.usc.edu/login.jsp?project=ADNIhttps://ida.loni.usc.edu/login.jsp?project=ADNI
 3. Go to Download > Image Collections > Advanced Search
 4. Copy and paste the list of subjects in "subjects.txt" (found in my GitHub repository) in the "Subject ID" field.
 5. In the "Research Group" section, select AD.
 6. In the "Study/Visit" section, select all the checkboxes that match "ADNI1/GO Month X," where X is a number.
 7. Check off "T1" in the "Weighting" section.
 8. Click "Search."
 9. Check "Select All" and click "Add to Collection."
 10. Enter a name for your collection and click "OK."
 11. Go to Data Collections.
 12. Click "My Collection" > *name of your collection* > "Not Downloaded"
 13. Check the "All" box.
 14. Click "1-Click Download"
 15. Click "Zip File 1" to download a zip file of the data.
 16. Unzip the file.
 17. Convert the images to JPG (see https://medium.com/@vivek8981/dicom-to-jpg-and-extract-all-patients-information-using-python-5e6dd1f1a07d for instructions.)
 18. Create a folder called "data", which will hold your JPG files, in Google Drive.
 19. Within that folder, create a folder called "train".
 20. Create a folder called "AD" within the "train" folder and move the images there.
 21. Repeat steps 3-17, this time with "CN" data (i.e., check the "CN" box in the "Research Group" section and create a folder called "CN" within your "train" folder in Google Drive that holds these images.)
 22. Create an empty "val" folder with empty "AD" and "CN" folders.
 23. In the notebook, change the variable called "root" to the path of your folder that includes the "data" folder (for instance, mine is '/content/drive/MyDrive/CS_598_Final_Project')


## Preprocessing

The paper states that "All longitudinal images per subject were used for training, whereas a single image, randomly selected from the longitudinal set, was selected for each validation and test patient."

To create the data loaders, I used all images for the training set, and randomly selected an image for each patient to create the validation set

### Create validation dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# TODO: change this to the name of your folder
root = '/content/drive/MyDrive/CS_598_Final_Project'

def create_validation_folder(root, p_type):
  files = os.listdir(os.fsencode(root + '/train/' + p_type))
  patients = set(list(map(lambda file: file[0:15], files)))

  for patient in patients:
    p = patient.decode()
    images = !ls $root/train/$p_type/$p*
    selected = random.sample(images, 1)[0]
    val_path = root + '/val/' + p_type

    !cp $selected $val_path

# create_validation_folder(root, 'AD')
# create_validation_folder(root, 'CN')

### Load data

In [ ]:
def load_data(root_path):
    t = transforms.Compose([
        transforms.RandomResizedCrop(size=(244, 244)),
        transforms.ToTensor()
    ])

    train_folder = torchvision.datasets.ImageFolder(
        root=root_path + '/train',
        transform=t
    )

    val_folder = torchvision.datasets.ImageFolder(
        root=root_path + '/val',
        transform=t
    )

    train_loader = torch.utils.data.dataloader.DataLoader(
        train_folder,
        batch_size=32,
        shuffle=True,
    )

    val_loader = torch.utils.data.dataloader.DataLoader(
        val_folder,
        batch_size=32,
        shuffle=False
    )

    return train_loader, val_loader

def get_stats(root):
    number_cn = !ls $root/CN | wc -l
    number_ad = !ls $root/AD | wc -l

    return int(number_cn[0]), int(number_ad[0])

train_loader, val_loader = load_data(root + '/data')
print('Train stats: ' + str(get_stats(root + '/data/train')))
print('Val stats: ' + str(get_stats(root + '/data/val')))

### Statistics

The training set includes 887 CN (control) images and 691 AD (Alzheimer's Disease) patients. The validation set includes 15 CN images and 14 AD images.

### Display images

Once you have your train loader, display the images along with their labels using these methods (taken from HW3 Convolutional Neural Network). I cannot show the images to others unless they also have access to the dataset.

In [ ]:
# Source: Homework 3: CNNs
def imshow(img, title):
    npimg = img.numpy()
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.show()

def show_batch_images(dataloader, k=8):
    images, labels = next(iter(dataloader))
    images = images[:k]
    labels = labels[:k]
    img = torchvision.utils.make_grid(images, padding=25)
    imshow(img, title=["AD" if x==0  else "CN" for x in labels])

'''
for i in range(2):
    show_batch_images(train_loader)
'''

## Model

### Original Paper
"Back to the Basics with Inclusion of Clinical Domain Knowledge - A Simple, Scalable and Effective Model of Alzheimer’s Disease Classification" (Sarah C. Brüningk, Felix Hensel, Louis P. Lukas, Merel Kuijs, Catherine R. Jutzeler, Bastian Rieck Proceedings of the 6th Machine Learning for Healthcare Conference, PMLR 149:730-754, 2021.)

### Original Repository
https://github.com/BorgwardtLab/ADNI_3DCNNvsTDA/blob/main

### Model Descriptions
I decided not to use the original repository code for this project. Instead, I created my own simple CNN model, using Homework 3 as a guide. In the paper, the authors said they used batch normalization, dropout, and ReLU activation for image-based classification, so I used these functions, as well.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.act = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.out = nn.Dropout()
        self.fc1 = nn.Linear(86400, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act(x)
        x = self.pool(x)
        x = self.out(x)
        x = torch.flatten(x, 1)
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.fc3(x)
        return x

model = CNN()

## Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

### Hyperparameters
The hyperparameters I chose are
- Learning rate (0.0001)
- Batch size (32)
- Dropout


### Computational Requirements
- **Number of epochs**: the authors used 2500, but I decided to use 10
- **Average runtime per epoch**: 1.36 minute
- **Type of hardware**: CPU

### Training Code

In [ ]:
criterion = torch.nn.modules.loss.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)

n = 10

def train_model(model, train_loader, n, optimizer, criterion):
    model.train()

    for epoch in range(n):
        for data, target in tqdm(train_loader):
            outputs = model(data)
            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Evaluate model and save state
'''
model = train_model(model, train_loader, n, optimizer, criterion)
torch.save(dict(model.state_dict()), root + '/model.pth')
'''

## Evaluation

### Code

In [ ]:
def evaluate(model, dataloader):
    model.eval()
    Y_pred = []
    Y_true = []

    for data, target in dataloader:
        pred = model(data).argmax(1).detach().numpy()
        true = list(target.detach().numpy())

        Y_pred.append(pred)
        Y_true.append(true)

    Y_pred = np.concatenate(Y_pred, axis=0)
    Y_true = np.concatenate(Y_true, axis=0)

    return Y_pred, Y_true

def load_saved_model(url, output, load, c):
  gdown.download(url, output)

  if c == 0:
    loaded_model = CNN()
  elif c == 1:
    loaded_model = Ablation1CNN()
  elif c == 2:
    loaded_model = Ablation2CNN()
  else:
    loaded_model = Ablation3CNN()

  loaded_model.load_state_dict(torch.load(load))
  return loaded_model

def get_metrics(model, val_loader):
  y_pred, y_true = evaluate(model, val_loader)

  acc = accuracy_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  roc_auc = roc_auc_score(y_true, y_pred)

  data = {
    'Metric': ['Accuracy', 'F1', 'Precision', 'Recall', 'ROC AUC'],
    'Result': [acc, f1, precision, recall, roc_auc]
  }

  df = pd.DataFrame(data=data)
  return df

url = 'https://drive.google.com/uc?id=1-ojSIoDlQuVm-KEmY_pWtyaGkFJzTACH'
output = 'nvswamy2-model'
load = '/content/nvswamy2-model'
loaded_model = load_saved_model(url, output, load, 0)

# Results

Since I used a much simpler model and technique than the one described in the paper, my numbers were different than the authors', who "initially used the full-resolution inner brain volume as input, then evaluated geometric brain subregions (image patches), and finally, brain functional subunits (left and right hippocampus). For each of these image subsets both a TDA and
purely image domain-based model was evaluated" (8). They also  used simple multilayer 3D CNNs with batch normalization, dropout, and ReLU activation for image-based classification. They then used convolutional layers and global
average pooling and two dense layers prior to the output layer. They also used four-layer 2D CNNs for persistence image inputs. Lastly, they used a simple GNN for patch images.

To evaluate the performance of their model, they used five-fold cross validation and split patients into test/test sets with an 80%-20% split. Within the training sets, they used a 75%-25% split to obtain the final training and validation sets.

They found that for the 3D images, their model had an accuracy and AUC of 0.79 +/ 0.05 and 0.88 +/- 0.05, respectively. When analyzing 3D image patches, their highest accuracy and AUC were 0.81 +/ 0.05 and 0.89 +/- 0.05, respectively (these numbers were observed if the patch was close to the hippocampus.) They also evaluated a 3D CNN model on the left and right hippocampus individually and found that the model on the left yieled better results than the right (0.84 +/ 0.07 accuracy and 0.91 +/- 0.05 AUC vs. 0.80 +/- 0.08 accuracy and 0.88 +/- 0.06 AUC).

In my version, my results are below:

In [ ]:
get_metrics(loaded_model, val_loader)

My accuracy and AUC (0.51, 0.5, respectively) are much lower than the results from the paper.

## Ablation Study

For the ablation study, I decided to remove the certain components from the CNN model to see how they impact performance.

### Removing Batch Normalization

In [ ]:
class Ablation1CNN(nn.Module):
    def __init__(self):
        super(Ablation1CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.act = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.out = nn.Dropout()
        self.fc1 = nn.Linear(86400, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.act(x)
        x = self.pool(x)
        x = self.out(x)
        x = torch.flatten(x, 1)
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.fc3(x)
        return x

'''
ablation_1_model = Ablation1CNN()
optimizer = torch.optim.Adam(params=ablation_1_model.parameters(), lr=0.0001)
ablation_1_model = train_model(ablation_1_model, train_loader, n, optimizer, criterion)
torch.save(dict(ablation_1_model.state_dict()), root + '/ablation-1-model.pth')
'''

url = 'https://drive.google.com/uc?id=1--NtUimicL534BDfWG789KH2AXGyygre'
output = 'nvswamy2-ablation-1-model'
load = '/content/nvswamy2-ablation-1-model'

loaded_ablation_model = load_saved_model(url, output, load, 1)
loaded_ablation_model.load_state_dict(torch.load(load))

get_metrics(loaded_ablation_model, val_loader)

### Removing ReLU Activation

In [ ]:
class Ablation2CNN(nn.Module):
    def __init__(self):
        super(Ablation2CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.out = nn.Dropout()
        self.fc1 = nn.Linear(86400, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool(x)
        x = self.out(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

'''
ablation_2_model = Ablation2CNN()
optimizer = torch.optim.Adam(params=ablation_2_model.parameters(), lr=0.0001)
ablation_2_model = train_model(ablation_2_model, train_loader, n, optimizer, criterion)
torch.save(dict(ablation_2_model.state_dict()), root + '/ablation-2-model.pth')
'''

url = 'https://drive.google.com/uc?id=1-4Aop_-iOSCUju39TGtRt7U8V8eAhPeg'
output = 'nvswamy2-ablation-2-model'
load = '/content/nvswamy2-ablation-2-model'

loaded_ablation_model = load_saved_model(url, output, load, 2)
loaded_ablation_model.load_state_dict(torch.load(load))

get_metrics(loaded_ablation_model, val_loader)

### Removing Dropout

In [ ]:
class Ablation3CNN(nn.Module):
    def __init__(self):
        super(Ablation3CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.act = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(86400, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.fc3(x)
        return x

'''
ablation_3_model = Ablation3CNN()
optimizer = torch.optim.Adam(params=ablation_3_model.parameters(), lr=0.0001)
ablation_3_model = train_model(ablation_3_model, train_loader, n, optimizer, criterion)
torch.save(dict(ablation_3_model.state_dict()), root + '/ablation-3-model.pth')
'''

url = 'https://drive.google.com/uc?id=1-4H-M19qHQMPCODpftXThAJQ5bKYJZQT'
output = 'nvswamy2-ablation-3-model'
load = '/content/nvswamy2-ablation-3-model'

loaded_ablation_model = load_saved_model(url, output, load, 3)
loaded_ablation_model.load_state_dict(torch.load(load))

get_metrics(loaded_ablation_model, val_loader)

Based on the results, it appears the the ReLU activation had the most impact since each metric increased once removing it, while removing batch normalization and dropout had almost no impact.

# Discussion

This project was difficult to duplicate for a couple of reasons.
1. While the original paper discusses the origin of the data and the metrics (sample size, mean age, etc.), they do not provide instructions on which specific images they used and how they navigated the ADNI database. I was able to find a list of subjects used from the original repository, so I downloaded images from a subset of those subjects.
2. The original code was quite complicated for me, as I have limited experience in working with neural networks and PyTorch. However, I understand that this is a skill in which I can improve and not entirely the fault of the authors.

I think the authors could improve their project by having a more through README with instructions on how to access the data and how to navigate the codebase. Comments throughout the code files would have been very helpful, as well.

If I were to improve my version of the project in the future, I would perform experiments on 3D images. I would also take patches of images and see how the model compares on the left versus the right hippocampus as the authors did.




# Public GitHub Link
https://github.com/nehaswamy/cs598-final-project/tree/main
